In [1]:
import pandas as pd
import numpy as np

In [2]:
# Before you doing this, you need to remove the first line(header) of the below file
match_best = pd.read_csv('Algorithm/match_best.csv', header=None)
match_second = pd.read_csv('Algorithm/match_second.csv', header=None)
match_third = pd.read_csv('Algorithm/match_third.csv', header=None)
raw_data = pd.read_csv('Algorithm/raw_data.csv', header=None)
Training_data = pd.read_csv('Algorithm/training_data.csv', header=None)
Testing_data = pd.read_csv('Algorithm/testing_data.csv', header=None)

similar_best = pd.read_csv('Algorithm/similar_best.csv', header=None)
similar_second = pd.read_csv('Algorithm/similar_second.csv', header=None)
similar_third = pd.read_csv('Algorithm/similar_third.csv', header=None)

In [3]:
# Training_data # 17583 rows × 117 columns

In [4]:
# similar_best 1959 rows × 116 columns

In [5]:
U = match_second - match_third
# U1 = similar_best - similar_second 
# U2 = similar_second - similar_third 

In [6]:
# U = U.append(U1)
# U = U.append(U2)

In [7]:
U = U.reset_index() # 5877 rows × 116 columns
U = U.drop('index', axis=1)

In [8]:
# U 5877 rows × 116 columns

In [9]:
# U # 1959 rows × 116 columns
numOfColumns = U.shape[1]

In [10]:
# INPORTANT : numOfColumns + 1 
data = np.identity( numOfColumns + 1 )   # with topic features

In [11]:
# data

In [12]:
I = pd.DataFrame(data=data[1:,1:])

In [13]:
# I

In [14]:
parameter_l = 0.01
parameter_u = 1.32
# 0.01,1.32,0.04
C = pd.concat([parameter_l * I, parameter_u * U],ignore_index=True)

In [15]:
# C

In [16]:
def S(x) :
    if x > -1 and x < 1:
        return x
    elif x >= 1 :
        return 1
    else :
        return -1

# test the S(x)
# print S(0)
# print S(0.5)
# print S(-0.7)
# print S(-2)
# print S(1.8)

In [17]:
random_num = np.random.uniform(-1, 1, size=numOfColumns)

In [18]:
w = pd.DataFrame(data=random_num)

In [19]:
# w # 116 rows × 1 columns

In [20]:
parameter_g = 0.04
#0.01,1.32,0.04
C_Dot_W = C.dot(w) / parameter_g

In [21]:
# C_Dot_W

In [22]:
A = C_Dot_W[0].apply(S)

In [23]:
# A

In [24]:
# C_T : C 's transpose
C_T = C.transpose()

In [25]:
C_T_Dot_A = 0.5 * C_T.dot(A)

In [26]:
# C_T_Dot_A

In [27]:
# Training_data # 17583 rows × 118 columns

In [28]:
# with topic features
X = Training_data.ix[0:, 0:numOfColumns-1]
Y = Training_data.ix[0:, numOfColumns]

In [29]:
# X

In [30]:
# Y 

In [31]:
X_Mul_Y = X.mul(Y, axis=0)

In [32]:
# X_Mul_Y

In [33]:
Sum_X_Mul_Y = X_Mul_Y.sum()

In [34]:
# Sum_X_Mul_Y

In [35]:
Target = Sum_X_Mul_Y - C_T_Dot_A

In [36]:
Target = Target.as_matrix()

In [37]:
# Target

In [38]:
X_T = X.transpose()

In [39]:
X_T_Dot_X = X_T.dot(X)

In [40]:
# X_T_Dot_X

In [41]:
Parameter = X_T_Dot_X.as_matrix()

In [42]:
# Parameter

In [43]:
w = np.linalg.solve(Parameter, Target)

In [44]:
# w

# Optimaization Algorithm

In [ ]:
iterations = 0
delta_A = 1000
delta_w = 1000
bound = 0.0001

while delta_A > bound and delta_w > bound :
    iterations += 1
    
    C_Dot_W = C.dot(w) / parameter_g
    
    # update vector A
    A_now = C_Dot_W.apply(S)
    
    C_T_Dot_A = 0.5 * C_T.dot(A_now)
    Target = Sum_X_Mul_Y - C_T_Dot_A
    Target = Target.as_matrix()
    
    # update vector w
    w_now = np.linalg.solve(Parameter, Target)
    
    delta_A = np.linalg.norm(A_now-A)
    delta_w = np.linalg.norm(w_now-w)
    
    A = A_now
    w = w_now
    
    print 'iterations:' + str(iterations) + ' ' + \
            'delta_A:' + str(delta_A) + ' ' + \
            'delta_w:' + str(delta_w)  
print w

iterations:1 delta_A:62.6428550283 delta_w:142789551.877
iterations:2 delta_A:47.0679248068 delta_w:273146256.282
iterations:3 delta_A:62.8386738848 delta_w:273153667.692
iterations:4 delta_A:63.0505174433 delta_w:273153768.339
iterations:5 delta_A:63.0550044385 delta_w:273153769.78
iterations:6 delta_A:63.0553649653 delta_w:273153769.95
iterations:7 delta_A:63.0553843689 delta_w:273153769.956
iterations:8 delta_A:63.0553864582 delta_w:273153769.957
iterations:9 delta_A:63.0553865629 delta_w:273153769.957
iterations:10 delta_A:63.0553865869 delta_w:273153769.957
iterations:11 delta_A:63.0553865834 delta_w:273153769.957
iterations:12 delta_A:63.0553865898 delta_w:273153769.957
iterations:13 delta_A:63.0553865853 delta_w:273153769.957
iterations:14 delta_A:63.0553865769 delta_w:273153769.957
iterations:15 delta_A:63.0553865927 delta_w:273153769.957
iterations:16 delta_A:63.0553866008 delta_w:273153769.957
iterations:17 delta_A:63.0553866041 delta_w:273153769.957
iterations:18 delta_A:63.

In [46]:
test_features = Testing_data.ix[0:, 0:numOfColumns-1]
test_flag = Testing_data.ix[0:, numOfColumns]

In [47]:
# Testing_data
# test_features

In [48]:
result = np.dot(test_features, w)

In [49]:
print result.max()
print result.min()
#print result
predict_flag = result > 0

2.42742283459
-2.40825894524


In [50]:
len(predict_flag)
print predict_flag[0]
if predict_flag[0] == 1:
    print "True == 1"

True
True == 1


In [51]:
#test_flag

true_positive = 0.0
true_negative = 0.0
false_positive = 0.0
false_negative = 0.0

for i in range( len(predict_flag) ):
    if predict_flag[i] == True and test_flag[i] == 1 :
        true_positive += 1
    if predict_flag[i] == False and test_flag[i] == -1 :
        true_negative += 1
    if predict_flag[i] == True and test_flag[i] == -1 :
        false_positive += 1
    if predict_flag[i] == False and test_flag[i] == 1 :
        false_negative += 1
        
#print true_positive
#print true_negative
#print false_positive
#print false_negative

ACC = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
PRE = true_positive / (true_positive + false_positive)
REC = true_positive / (true_positive + false_negative)
F1 = 2*( PRE * REC) / (PRE + REC)
print ACC
print PRE
print REC
print F1

0.941017802553
0.940684133916
0.94
0.940341942525


In [52]:
def Store_Model( Weights , File ) :
    # Write Model's performance to the File
    File.write(str(F1*100) + '%')
    
    # Write coefficients to the Filse 
    coefficients = Weights
    for i in range( len(coefficients) ) :
        # print i,coefficients[i]
        File.write(',' + str(coefficients[i]) )

In [53]:
with open('./MODEL/MODEL_03.txt', 'w') as model :
    Store_Model(w, model)